<a href="https://colab.research.google.com/github/SrtaCamelo/nodule_detection/blob/main/nodule_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nodule Detection on LUNA images using UNET variations

In [ ]:
!git clone https://github.com/SrtaCamelo/nodule_detection


fatal: destination path 'nodule_detection' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
